# visualizing termination time versus domain size
Tim Tyree<br>
6.11.2025

In [1]:
import os,sys,pandas as pd,numpy as np

In [2]:
from lib import *

# define module

In [3]:
# parse_iolines
def parse_iolines_single(input_fn, printing=False):
    n_input=-9999
    n_output=-9999
    with open(input_fn) as f:
#         trgt1='Printing Inputs...\n'
#         trgt2='Printing Outputs...\n'
        trgt1='Printing Inputs:\n'
        trgt2='Printing Outputs:\n'
        for n,line in enumerate(f):
            if trgt1 == line:
                if printing:
                    print(f'found inputs starting after line {n}')
                n_input=n
            if trgt2 == line:
                if printing:
                    print(f'found outputs starting after line {n}')
                n_output=n
    return n_input,n_output

In [4]:
def parse_outputs_single_trial(input_fn, n_output):
    """
    Example Usage:
TermTime = parse_outputs_single_trial(input_fn, n_output)
    """
    with open(input_fn) as f:
        for n,line in enumerate(f):
            if n == n_output+1:
                lin0 =line
                TermTime = eval(lin0[:-2].split('=')[-1])
                return TermTime
#         if n == n_output+1:
#             line_N = line
#         if n == n_output+2:
#             line_CollTime =line
# N_values=np.array([eval(s) for s in line_N[:-2].split(',')])
# CollTime_values=np.array([eval(s) for s in line_CollTime[:-2].split(',')])
# # CollTime_values=np.array([eval(s) for s in line_CollTime[:-1].split(',')])
# df=pd.DataFrame({
#     'N':N_values,
#     'CollTime':CollTime_values
# })
# df=df.loc[df.CollTime!=-9999]
# df.reset_index(inplace=True,drop=True)
## return df

In [5]:
# parse_outputs(input_fn, n_output=)
def parse_log_single(input_fn,printing=False):
    """
    Example Usage:
dict_data = parse_log_single(input_fn,printing=False)
    """
    n_input,n_output=parse_iolines_single(input_fn, printing=False)
    if n_input==-9999:
        if printing:
            print(f"Warning: n_input not found for input_fn={input_fn}.\nreturning None...")
        return None
    col_name_lst, col_value_lst=parse_inputs(input_fn,n_input,n_output,printing=printing)
    dict_inputs=dict(zip(col_name_lst,col_value_lst))
    TermTime = parse_outputs_single_trial(input_fn, n_output)
    dict_data = dict(dict_inputs)
    dict_data['TermTime'] = TermTime
    return dict_data

# dev

In [6]:
run_num = 48
# fldr = f"{nb_dir}/data/osg_output/run_44"
# fldr = f"{nb_dir}/data/osg_output/run_45"
# fldr = f"{nb_dir}/data/osg_output/run_46"
# fldr = f"{nb_dir}/data/osg_output/run_47"
fldr = f"{nb_dir}/data/osg_output/run_{run_num}"

In [7]:
assert os.path.exists(fldr)

In [8]:
dict_data_lst = []
for fn in os.listdir(fldr):
    if fn.find('.out.')!=-1:
        input_fn = os.path.join(fldr,fn)
        dict_data = parse_log_single(input_fn,printing=False)
        if dict_data is not None:
            #record
            dict_data_lst.append(dict_data)
df=pd.DataFrame(dict_data_lst); del dict_data_lst

In [9]:
# np.array([x for x in dict_data_lst if x is None])

In [10]:
# pd.DataFrame(dict_data_lst)

In [11]:
# len(dict_data_lst)

In [12]:
# input_fn

In [13]:
# !atom '/Users/timothytyree/Documents/GitHub/bgmc/python/data/osg_output/run_45/job.out.17477944.1912'

In [14]:
# dict_data = parse_log_single(input_fn,printing=True)
# dict_data

In [15]:
np.random.seed(42)
printing=True
if printing:
    print(f"LPM with Same Creation (Run #46):")
dict_lst = []
for (D,L),g in df.groupby(by=['D','L']):
    tau_values = g['TermTime'].values
    meanx,Delta_meanx,num_obs,p_normal=comp_mean_bootstrap_uncertainty(tau_values)
    printing=True
    a=g['varkappa'].max()
    chi=g['chi'].max()
    if printing:
        print(f"- {D=}, {L=}: mean tau: {meanx:.4f} +/- {Delta_meanx:.4f} seconds (N={num_obs}, {p_normal=:.4f})")
    dict_lst.append(dict(D=D,L=L,a=a,chi=chi,
                         tau=meanx,Delta_tau=Delta_meanx,
                         num_obs=num_obs,p_normal=p_normal,
                        ))

LPM with Same Creation (Run #46):
- D=0.115, L=3.75: mean tau: 2.5173 +/- 0.1073 seconds (N=2000, p_normal=0.5600)
- D=0.115, L=4.5: mean tau: 14.5283 +/- 7.3355 seconds (N=2000, p_normal=0.0000)
- D=0.115, L=4.75: mean tau: 29.2283 +/- 11.8199 seconds (N=2000, p_normal=0.0000)
- D=0.115, L=5.0: mean tau: 53.8336 +/- 17.2470 seconds (N=2000, p_normal=0.0000)
- D=0.115, L=5.25: mean tau: 70.9511 +/- 15.6924 seconds (N=2000, p_normal=0.0552)
- D=0.115, L=5.625: mean tau: 166.4115 +/- 27.6704 seconds (N=2000, p_normal=0.0342)
- D=0.115, L=6.25: mean tau: 624.0579 +/- 49.0198 seconds (N=2000, p_normal=0.4734)
- D=0.115, L=7.075: mean tau: 2603.9936 +/- 96.3658 seconds (N=2000, p_normal=0.9836)
- D=0.115, L=8.75: mean tau: 5656.2152 +/- 49.9363 seconds (N=1996, p_normal=0.1367)
- D=0.42, L=5.0: mean tau: 1.6145 +/- 2.1266 seconds (N=2000, p_normal=0.0000)
- D=0.42, L=6.25: mean tau: 2.0326 +/- 0.8127 seconds (N=2000, p_normal=0.0000)
- D=0.42, L=7.5: mean tau: 15.9232 +/- 7.1594 seconds (N=

In [16]:
df_out = pd.DataFrame(dict_lst)
df_out

,D,L,a,chi,tau,Delta_tau,num_obs,p_normal
0,0.115,3.750,1.552,2.939433,2.517330,0.107323,2000,5.600150e-01
1,0.115,4.500,1.552,2.939433,14.528281,7.335516,2000,3.267337e-18
2,0.115,4.750,1.552,2.939433,29.228292,11.819934,2000,3.426592e-10
3,0.115,5.000,1.552,2.939433,53.833603,17.246971,2000,8.574206e-08
4,0.115,5.250,1.552,2.939433,70.951114,15.692392,2000,5.523454e-02
5,0.115,5.625,1.552,2.939433,166.411489,27.670420,2000,3.418187e-02
6,0.115,6.250,1.552,2.939433,624.057903,49.019808,2000,4.734398e-01
7,0.115,7.075,1.552,2.939433,2603.993633,96.365823,2000,9.836285e-01
8,0.115,8.750,1.552,2.939433,5656.215250,49.936347,1996,1.367344e-01
9,0.420,5.000,9.300,2.075269,1.614531,2.126633,2000,3.545739e-33


In [17]:
# savedir = f"{nb_dir}/data/osg_output/run_46_tau_vs_L.csv"
# savedir = f"{nb_dir}/data/osg_output/run_47_tau_vs_L.csv"
savedir = f"{nb_dir}/data/osg_output/run_{run_num}_tau_vs_L.csv"

In [18]:
df_out.to_csv(savedir,index=False)
print(savedir)

/Users/timothytyree/Documents/GitHub/bgmc/python/data/osg_output/run_48_tau_vs_L.csv


In [19]:
beep(3)

-n 
-n 
-n 


0